## Resource Catalogue demo

[OWSLib](https://geopython.github.io/OWSLib) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. In this demo we’ll work with the CSW, WMS and WCS interfaces.

In [ ]:
from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link
import folium
import json

In [ ]:
base_domain = "185.52.193.87.nip.io"
#base_domain = "demo.eoepca.org"
#workspace_prefix = "demo-user"
workspace_prefix = "user"

### System Catalogue Discovery

In this part of the demo, the user will use the system level resource catalogue endpoint to discover data collections and datasets.
The `owslib.csw` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}/csw'

In [ ]:
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

Service metadata shown here includes identification type (from ISO-19115), CSW version and supported operations

In [ ]:
csw.identification.type

In [ ]:
csw.version

In [ ]:
[op.name for op in csw.operations]

As well as catalogue queryables:

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

The user can make a GetRecords request to get all records of the catalogue, with a page limit of 10.

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

If the user wishes to discover data with usage of filters, an OGC Filter can be used. Here we demonstrate how to create spatial (`bbox`), temporal (`time`), and attribute (`apiso:CloudCover`) filters combined with logical operators like and/or

In [ ]:
#bbox_query = BBox([37.8, 23.4, 38.8, 24.5])
bbox_query = BBox([37, 13.9, 37.9, 15.1])

In [ ]:
begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2021-04-02 00:00')

In [ ]:
end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2021-04-03 00:00')

In [ ]:
cloud = PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            begin, end,  # start and end date
            cloud        # cloud
        ]
    )
]

The filter is then applied to the GetRecords request and results are shown:

In [ ]:
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
selected_record = list(csw.records)[0]

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

Another option is to perform a collection level search, using the `apiso:parentIdentifier` queryable. Here only the Sentinel2 L1C datasets will be discovered.

In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'S2MSI1C')

In [ ]:
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

Or we can just search using the bbox filter

In [ ]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

Or we can perform a full text search (here the keyword Orthoimagery is used)

In [ ]:
anytext_query = PropertyIsEqualTo('csw:AnyText', 'Orthoimagery')

In [ ]:
csw.getrecords2(constraints=[anytext_query])
csw.results

We can also iterate through the catalogue search results by passing the `startposition` and `maxrecords` parameters to GetRecords request:

In [ ]:
csw_records = {}
sortby = SortBy([SortProperty('dc:title', 'ASC')])
pagesize=10
maxrecords=1000
startposition = 0
nextrecord = getattr(csw, 'results', 1)
while nextrecord != 0:
    csw.getrecords2(constraints=[anytext_query], startposition=startposition,
                    maxrecords=pagesize, sortby=sortby)
    csw_records.update(csw.records)
    if csw.results['nextrecord'] == 0:
        break
    startposition += pagesize
    if startposition >= maxrecords:
        break
csw.records.update(csw_records)
records = '\n'.join(csw.records.keys())
print('Found {} records.\n'.format(len(csw.records.keys())))
for key, value in list(csw.records.items()):
    print(f'identifier: {value.identifier}\ntype: {value.type}\ntitle: {value.title}\n')

The user then selects a record identifier and asks the catalogue to fetch the full record. Here we demonstrate how to obtain properties like title, bbox, full xml and links from the metadata record.

In [ ]:
#csw.getrecordbyid(id=['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'])
csw.getrecordbyid(id=[selected_record])

In [ ]:
#rec = csw.records['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE']
rec = csw.records[selected_record]

In [ ]:
rec.title

In [ ]:
rec.xml

In [ ]:
rec.references

In [ ]:
print("dataset bbox = (%s, %s, %s, %s)" % (rec.bbox.miny, rec.bbox.minx, rec.bbox.maxy, rec.bbox.maxx))

Using the [geolinks](https://github.com/geopython/geolinks) Python library we can filter the links that are of a specific type (here WMS and WCS links to be used for visualization)

In [ ]:
msg = 'geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n'.format
for ref in rec.references:
    print(msg(geolink=sniff_link(ref['url']), **ref))

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WMS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WCS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

Finally we demonstrate how to show the record footprint on a map, using the [Folium](https://github.com/python-visualization/folium) Python library:

In [ ]:
m = folium.Map(location=[38, 20], zoom_start=6, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(rec.bbox.miny), float(rec.bbox.minx)], [float(rec.bbox.maxy), float(rec.bbox.maxx)]]).add_to(m)
m

### Workspace catalogue

In this part of the demo, the user will use the workspace resource catalogue endpoint to discover processing outputs and applications.  The workspace catalogue is local to the user, which also has federated access to the system catalogue.

In [ ]:
#workspace_endpoint = 'https://resource-catalogue.user.185.52.193.87.nip.io'
#workspace_endpoint = 'https://resource-catalogue.user.demo.eoepca.org'
#workspace_endpoint = 'https://resource-catalogue.rm-user-eric.185.52.193.87.nip.io'
workspace_endpoint = f'https://resource-catalogue.{workspace_prefix}.{base_domain}/csw'

In [ ]:
csw = CatalogueServiceWeb(workspace_endpoint,timeout=30)

With a GetRecords request the user can see the number of total records in the workspace catalogue

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

Then, using the identifiers, the user can parse the links to the output data as well as any applications that are available ([Common Workflow Language [CWL]](https://www.commonwl.org))

In [ ]:
csw.records['INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'].references

In [ ]:
csw.records['s-expression'].references

In [ ]:
csw.getrecords2(maxrecords=20, distributedsearch=True, hopcount=1)
csw.results

### OpenSearch

In this part of the demo, the user will use the OpenSearch capability of the system resource catalogue to discover datasets.

In [ ]:
endpoint = system_catalogue_endpoint + '?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'

Here, the [pyops](https://github.com/devsf/pyops) Python OpenSearch client is used.

In [ ]:
import pyops

In [ ]:
client = pyops.Client(description_xml_url=endpoint)

and the number of results is shown (note that the page size is 10 by default)

In [ ]:
results = client.search()
len(results)

The user can also include parameters in the search, like the parent identifier for collection level search

In [ ]:
results = client.search(params={"{eo:parentIdentifier?}": {"value": "S2MSI2A"}})
len(results)

The user can also use OpenSearch EO mathematical notation to filter based on other parameters

In [ ]:
# OpenSearch EO mathematical notation
# n1 equal to field = n1
# {n1,n2,…} equals to field=n1 OR field=n2 OR …
# [n1,n2] equal to n1 <= field <= n2
# [n1,n2[ equals to n1 <= field < n2
# ]n1,n2[ equals to n1 < field < n2
# ]n1,n2] equal to n1 < field  <= n2
# [n1 equals to n1<= field
# ]n1 equals to n1 < field
# n2] equals to field <= n2
# n2[ equals to field < n2

results = client.search(params={"{eo:cloudCover?}": {"value": "]20"}})  # eo:cloudCover > 20
len(results)

Then the user can access a specific record through a list (with a JSON representation):

In [ ]:
results[1]

Another posibility is to use the HTTP GET requests directly with a generic client like requests.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
S = requests.Session()

Here, the user asks for the OpenSearch entrypoint template

In [ ]:
R = S.get(url=endpoint)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The a GetRecords request (in the context of OpenSearch) is performed

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A collections search is also demonstrated, for Sentinel2 Level 2A results

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:parentIdentifier=S2MSI2A'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A similar OpenSearch request is demonstrated for the workspace catalogue endpoint

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can get one record using the identifier through the OpenSearch EO API:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&recordids=S2B_MSIL1C_20210402T095029_N0300_R079_T33SVB_20210402T121737.SAFE'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Finally, both system and workspace endpoints can be queried using a Federated Search query, where the user sends a request to the workspace catalogue which in turn sends a distributed search request to the system catalogue and aggregates the results for the user.

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&typenames=csw:Record&elementsetname=full&resulttype=results&distributedSearch=TRUE&hopcount=1'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Please note in the above result that the total of records is a combination of the system and workspace catalogues.

### OGC API Records demo

In this part of the demo, the user will use the system level resource catalogue OGC API Records endpoint to discover data datasets.
The `owslib.ogcapi.records` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'

In [ ]:
w = Records(system_catalogue_endpoint)

In [ ]:
w.url

Conformance classes supported by the OGC API Records server:

In [ ]:
w.conformance()

OpenAPI document of the OGC API Records server:

In [ ]:
w.api()

Collections available on the catalogue:

In [ ]:
w.collections()

In [ ]:
records = w.records()

In [ ]:
len(records)

The user can then specify the collection to search within the catalogue:

In [ ]:
my_catalogue = w.collection('metadata:main')

In [ ]:
my_catalogue['id']

Collection level queryables:

In [ ]:
w.collection_queryables('metadata:main')

The user initialized a catalogue query for the specific collection:

In [ ]:
my_catalogue_query = w.collection_items('metadata:main')

In [ ]:
my_catalogue_query['numberMatched']

In [ ]:
my_catalogue_query['features'][0]['properties'].keys()

In [ ]:
my_catalogue_query['features'][0]['properties']['title']

In [ ]:
my_catalogue_query2 = w.collection_items('metadata:main', q='Orthoimagery')

In [ ]:
my_catalogue_query2['numberMatched']

Full query result:

In [ ]:
my_catalogue_query2

### STAC API demo

In [ ]:
from pystac_client import Client
from pystac_client import ConformanceClasses

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'

In [ ]:
catalog = Client.open(system_catalogue_endpoint)

In [ ]:
catalog.id

In [ ]:
catalog.title

In [ ]:
catalog.description

In [ ]:
dir(ConformanceClasses)

In [ ]:
catalog.assert_conforms_to(ConformanceClasses.ITEM_SEARCH)

In [ ]:
catalog.assert_conforms_to(ConformanceClasses.CORE)

In [ ]:
mysearch = catalog.search(collections=['metadata:main'], max_items=10)
#mysearch = catalog.search(collections=['metadata:main'], bbox=[-72.5,40.5,-72,41], max_items=10)
#mysearch = catalog.search(collections=['metadata:main'], bbox=[37, 13.9, 37.9, 15.1], max_items=10)
print(f"{mysearch.matched()} items found")

In [ ]:
items = mysearch.get_items()
for item in items:
    print(item.id)
    #print(item.to_dict())
    #print(json.dumps(item.to_dict(), indent=2))

### QGIS catalogue demo

In this part of the demo, the user will use the system level resource catalogue endpoint to discover and visualize datasets through QGIS desktop application.

![QGIS main window with OSM loaded](img/Screenshot_QGIS_01.png)

The user starts up QGIS and loads some data, in this case the OSM base map. The MetaSearch tool is available on the toolbar

![MetaSearch main window](img/Screenshot_QGIS_02.png)

The user can add a new catalogue endpoint by pressing the New button, then needs to add the Resource Catalogue endpoint in the URL text box

![EOEPCA Resource Catalogue in QGIS](img/Screenshot_QGIS_03.png)

The service metadata for the Resource Catalogue are available from the Service Info button

![Service Info](img/Screenshot_QGIS_04.png)

The Service Capabilities are also available from the 'GetCapabilities Response' button

![Service Capabilities](img/Screenshot_QGIS_05.png)

The user moves to the 'Search' tab of MetaSearch main window to perform a catalogue search (in this case by adding a bounding box)

![Search with bbox](img/Screenshot_QGIS_06.png)

By pressing 'Search' the Resource Catalogue is performing a dataset search. Then by selecting a result, the dataset bbox is shown on the map.

![Search for datasets](img/Screenshot_QGIS_07.png)

The user can choose to view the selected dataset by selecting the 'Add Data' button, where MetaSearch automatically discovers available WMS, WFS, WCS links and enables the capability to load the data directly to QGIS map. In this case, the WMS and WCS endpoints are available from the Data Access links included in the catalogue record.

![Adding Data](img/Screenshot_QGIS_08.png)

By selecting 'Add WMS/WMTS' the default QGIS WMS dialog shows up. Here, the user can browse through the available layers offered by the Data Access component and select a layer to add to the map.

![QGIS WMS dialog](img/Screenshot_QGIS_09.png)

After selecting a layer, QGIS will add the layer to the map and user can preview the selected dataset

![Data preview](img/Screenshot_QGIS_10.png)

By double clicking the catalogue record, the user can also preview the record metadata

![Metadata preview](img/Screenshot_QGIS_11.png)